In [1]:
# https://keras.io/
!pip install -q keras
import keras

Using TensorFlow backend.


# New Section

In [1]:
import keras
from keras.datasets import cifar10
from keras.models import Model, Sequential
from keras.layers import Dense, Dropout, Flatten, Input, AveragePooling2D, merge, Activation
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization, GlobalAveragePooling2D
from keras.layers import Concatenate
from keras.optimizers import Adam

Using TensorFlow backend.


In [2]:
# this part will prevent tensorflow to allocate all the avaliable GPU Memory
# backend
import tensorflow as tf
from keras import backend as k

# Don't pre-allocate memory; allocate as-needed
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

# Create a session with the above options specified.
k.tensorflow_backend.set_session(tf.Session(config=config))

In [5]:
# Hyperparameters
batch_size = 128
num_classes = 10
epochs = 50
l = 45
num_filter = 12
compression = 0.7
dropout_rate = 0.4

In [4]:
# Load CIFAR10 Data
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
img_height, img_width, channel = x_train.shape[1],x_train.shape[2],x_train.shape[3]

# convert to one hot encoing 
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

170500096/170498071 [==============================] - 16s 0us/step


In [6]:
# Dense Block
def add_denseblock(input, num_filter = 21, dropout_rate = 0.2):
    global compression
    temp = input
    for _ in range(l):
        BatchNorm = BatchNormalization()(temp)
        relu = Activation('relu')(BatchNorm)
        Conv2D_3_3 = Conv2D(int(num_filter*compression), (3,3), use_bias=False ,padding='same')(relu)
        #Conv2D_3_3_1 = Conv2D(int(num_filter*compression), (3,3), use_bias=False ,padding='same')(Conv2D_3_3)
        if dropout_rate>0:
          Conv2D_3_3 = Dropout(dropout_rate)(Conv2D_3_3)
        concat = Concatenate(axis=-1)([temp,Conv2D_3_3])
        
        temp = concat
        
    return temp

In [7]:
def add_transition(input, num_filter = 21, dropout_rate = 0.2):
    global compression
    BatchNorm = BatchNormalization()(input)
    relu = Activation('relu')(BatchNorm)
    Conv2D_BottleNeck = Conv2D(int(num_filter*compression), (1,1), use_bias=False ,padding='same')(relu)
    #Conv2D_BottleNeck_1 = Conv2D(int(num_filter*compression), (1,1), use_bias=False ,padding='same')(Conv2D_BottleNeck)
    if dropout_rate>0:
      Conv2D_BottleNeck = Dropout(dropout_rate)(Conv2D_BottleNeck)
    avg = AveragePooling2D(pool_size=(2,2))(Conv2D_BottleNeck)
    
    return avg

In [8]:
def output_layer(input):
    global compression
    BatchNorm = BatchNormalization()(input)
    relu = Activation('relu')(BatchNorm)
    AvgPooling = AveragePooling2D(pool_size=(2,2))(relu)
    flat = GlobalAveragePooling2D()(AvgPooling)
    output = Dense(num_classes, activation='softmax')(flat)
    
    return output

In [9]:
num_filter = 21
dropout_rate = 0.4
l = 12
input = Input(shape=(img_height, img_width, channel,))
First_Conv2D = Conv2D(num_filter, (3,3), use_bias=False ,padding='same')(input)
First_Conv2D = Conv2D(num_filter, (3,3), use_bias=False ,padding='same')(First_Conv2D)

First_Block = add_denseblock(First_Conv2D, num_filter, dropout_rate)
First_Transition = add_transition(First_Block, num_filter, dropout_rate)

Second_Block = add_denseblock(First_Transition, num_filter, dropout_rate)
Second_Transition = add_transition(Second_Block, num_filter, dropout_rate)

Third_Block = add_denseblock(Second_Transition, num_filter, dropout_rate)
Third_Transition = add_transition(Third_Block, num_filter, dropout_rate)

Last_Block = add_denseblock(Third_Transition,  num_filter, dropout_rate)
output = output_layer(Last_Block)


In [10]:
model = Model(inputs=[input], outputs=[output])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 32, 32, 21)   567         input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_2 (Conv2D)               (None, 32, 32, 21)   3969        conv2d_1[0][0]                   
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 32, 32, 21)   84          conv2d_2[0][0]                   
__________________________________________________________________________________________________
activation

In [11]:
from keras.callbacks import ModelCheckpoint
checkpoint=ModelCheckpoint('weights.best.hdf5', save_best_only=True, verbose=1)
# determine Loss function and Optimizer
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(lr=0.01),
              metrics=['accuracy'])

In [12]:
model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_test, y_test), callbacks=[checkpoint])

Train on 50000 samples, validate on 10000 samples
Epoch 1/50
50000/50000 [==============================] - 341s 7ms/step - loss: 1.6687 - acc: 0.3706 - val_loss: 2.0828 - val_acc: 0.4037
Epoch 2/50
50000/50000 [==============================] - 325s 6ms/step - loss: 1.2892 - acc: 0.5266 - val_loss: 1.6734 - val_acc: 0.5088
Epoch 3/50
50000/50000 [==============================] - 324s 6ms/step - loss: 1.0893 - acc: 0.6090 - val_loss: 1.7531 - val_acc: 0.4864
Epoch 4/50
50000/50000 [==============================] - 325s 6ms/step - loss: 0.9840 - acc: 0.6473 - val_loss: 1.3377 - val_acc: 0.5778
Epoch 5/50
50000/50000 [==============================] - 325s 6ms/step - loss: 0.9211 - acc: 0.6725 - val_loss: 1.2069 - val_acc: 0.5962
Epoch 6/50
50000/50000 [==============================] - 324s 6ms/step - loss: 0.8563 - acc: 0.6958 - val_loss: 1.4154 - val_acc: 0.5662
Epoch 7/50
50000/50000 [==============================] - 325s 6ms/step - loss: 0.8099 - acc: 0.7128 - val_loss: 1.1592 - 

Epoch 31/50
50000/50000 [==============================] - 325s 6ms/step - loss: 0.4523 - acc: 0.8413 - val_loss: 0.7487 - val_acc: 0.7401
Epoch 32/50
50000/50000 [==============================] - 325s 7ms/step - loss: 0.4516 - acc: 0.8426 - val_loss: 0.7093 - val_acc: 0.7504
Epoch 33/50
50000/50000 [==============================] - 325s 6ms/step - loss: 0.4413 - acc: 0.8474 - val_loss: 0.7887 - val_acc: 0.7242
Epoch 34/50
50000/50000 [==============================] - 325s 7ms/step - loss: 0.4398 - acc: 0.8474 - val_loss: 0.6921 - val_acc: 0.7657
Epoch 35/50
50000/50000 [==============================] - 325s 7ms/step - loss: 0.4325 - acc: 0.8496 - val_loss: 0.6338 - val_acc: 0.7830
Epoch 36/50
50000/50000 [==============================] - 325s 6ms/step - loss: 0.4298 - acc: 0.8511 - val_loss: 1.0384 - val_acc: 0.6622
Epoch 37/50
50000/50000 [==============================] - 325s 6ms/step - loss: 0.4170 - acc: 0.8545 - val_loss: 0.7782 - val_acc: 0.7323
Epoch 38/50
50000/50000 [==

In [14]:
# Test the model
model.load_weights('weights.best.hdf5')
score = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

10000/10000 [==============================] - 29s 3ms/step
Test loss: 0.588484449387
Test accuracy: 0.7933


In [15]:
# Save the trained weights in to .h5 format
model.save_weights("DNST_model.h5")
print("Saved model to disk")

Saved model to disk


In [0]:
from google.colab import files

files.download('DNST_model.h5')